In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="Haberman"
JobNameAbbrev = "HM"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                    #    "Seed":list(range(0,100)),
                       "Seed":list([]),                       
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.11],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["0:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["30000M"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Passive Learning

In [3]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(
[1, 4, 6, 7, 10, 11, 16, 20, 21, 22, 23, 26, 28, 29, 33, 34, 35, 37, 38, 41, 44, 45, 47, 51, 52, 54, 55, 57, 58, 59, 61, 63, 70, 71, 72, 74, 75, 77, 83, 85, 88, 89, 91, 93, 95, 98]
                           ),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["PassiveLearning"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

/var/folders/10/2lfzfs1j0j933_mjkrskp5kh0000gq/T/ipykernel_25831/217173303.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline


# Include/exclude Random Forest Simulations

In [4]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(
[1, 4, 6, 7, 10, 11, 16, 20, 21, 22, 23, 26, 28, 29, 33, 34, 35, 37, 38, 41, 44, 45, 47, 51, 52, 54, 55, 57, 58, 59, 61, 63, 70, 71, 72, 74, 75, 77, 83, 85, 88, 89, 91, 93, 95, 98]
                           ),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [5]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI1_", "_PL", regex=True)
    )

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

---

# Only run these ones

In [6]:
# # ### Simulations that failed ###
# FilterText = ['0HM_DA0.11', '2HM_DA0.11', '3HM_DA0.11', '5HM_DA0.11', '6HM_DA0.11', '8HM_DA0.11', '9HM_DA0.11', '10HM_DA0.11', '12HM_DA0.11', '13HM_DA0.11', '14HM_DA0.11', '15HM_DA0.11', '17HM_DA0.11', '18HM_DA0.11', '19HM_DA0.11', '21HM_DA0.11', '23HM_DA0.11', '24HM_DA0.11', '25HM_DA0.11', '27HM_DA0.11', '28HM_DA0.11', '29HM_DA0.11', '30HM_DA0.11', '31HM_DA0.11', '32HM_DA0.11', '33HM_DA0.11', '34HM_DA0.11', '36HM_DA0.11', '37HM_DA0.11', '39HM_DA0.11', '40HM_DA0.11', '42HM_DA0.11', '43HM_DA0.11', '46HM_DA0.11', '48HM_DA0.11', '49HM_DA0.11', '50HM_DA0.11', '53HM_DA0.11', '56HM_DA0.11', '60HM_DA0.11', '62HM_DA0.11', '63HM_DA0.11', '64HM_DA0.11', '65HM_DA0.11', '66HM_DA0.11', '67HM_DA0.11', '68HM_DA0.11', '69HM_DA0.11', '71HM_DA0.11', '73HM_DA0.11', '76HM_DA0.11', '78HM_DA0.11', '79HM_DA0.11', '80HM_DA0.11', '81HM_DA0.11', '82HM_DA0.11', '83HM_DA0.11', '84HM_DA0.11', '86HM_DA0.11', '87HM_DA0.11', '90HM_DA0.11', '92HM_DA0.11', '94HM_DA0.11', '95HM_DA0.11', '96HM_DA0.11', '97HM_DA0.11', '99HM_DA0.11', '0HM_UA0.11', '2HM_UA0.11', '3HM_UA0.11', '5HM_UA0.11', '8HM_UA0.11', '9HM_UA0.11', '10HM_UA0.11', '12HM_UA0.11', '13HM_UA0.11', '14HM_UA0.11', '15HM_UA0.11', '17HM_UA0.11', '18HM_UA0.11', '19HM_UA0.11', '24HM_UA0.11', '25HM_UA0.11', '28HM_UA0.11', '29HM_UA0.11', '30HM_UA0.11', '31HM_UA0.11', '32HM_UA0.11', '33HM_UA0.11', '34HM_UA0.11', '36HM_UA0.11', '37HM_UA0.11', '39HM_UA0.11', '40HM_UA0.11', '42HM_UA0.11', '43HM_UA0.11', '44HM_UA0.11', '46HM_UA0.11', '48HM_UA0.11', '49HM_UA0.11', '50HM_UA0.11', '53HM_UA0.11', '56HM_UA0.11', '60HM_UA0.11', '62HM_UA0.11', '63HM_UA0.11', '64HM_UA0.11', '65HM_UA0.11', '66HM_UA0.11', '67HM_UA0.11', '68HM_UA0.11', '69HM_UA0.11', '71HM_UA0.11', '73HM_UA0.11', '78HM_UA0.11', '79HM_UA0.11', '80HM_UA0.11', '81HM_UA0.11', '82HM_UA0.11', '83HM_UA0.11', '84HM_UA0.11', '86HM_UA0.11', '87HM_UA0.11', '90HM_UA0.11', '92HM_UA0.11', '94HM_UA0.11', '96HM_UA0.11', '97HM_UA0.11', '99HM_UA0.11']

# ### Rerun these simulations ###
# ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector

---

In [7]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,Partition,Time,Memory,JobName,Output
0,Haberman,1,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,1HM_PLA0,Haberman/RandomForestClassification/Raw/1HM_PL...
1,Haberman,1,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,1HM_RFA0,Haberman/RandomForestClassification/Raw/1HM_RF...
2,Haberman,4,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,4HM_RFA0,Haberman/RandomForestClassification/Raw/4HM_RF...
3,Haberman,4,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,4HM_PLA0,Haberman/RandomForestClassification/Raw/4HM_PL...
4,Haberman,6,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,6HM_PLA0,Haberman/RandomForestClassification/Raw/6HM_PL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Haberman,93,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,93HM_PLA0,Haberman/RandomForestClassification/Raw/93HM_P...
88,Haberman,95,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,95HM_RFA0,Haberman/RandomForestClassification/Raw/95HM_R...
89,Haberman,95,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,95HM_PLA0,Haberman/RandomForestClassification/Raw/95HM_P...
90,Haberman,98,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,98HM_PLA0,Haberman/RandomForestClassification/Raw/98HM_P...


# Save Parameter Vector

In [8]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)